This assignment will help you practice web scraping techniques by extracting structured data
from a live practice website. You will learn how to navigate HTML structures, extract relevant
information, and save it in a structured format for analysis.
Q1. Write a Python program to scrape all available books from the website
(https://books.toscrape.com/) Books to Scrape – a live site built for practicing scraping (safe,
legal, no anti-bot). For each book, extract the following details:

1. Title
2. Price
3. Availability (In stock / Out of stock)
4. Star Rating (One, Two, Three, Four, Five)
Store the scrap

In [6]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
response = requests.get("https://books.toscrape.com/")
html = response.text
print(response.status_code)
print(response.headers)
def scrap_books(soup):
  books_scrap = soup.find_all("article", class_="product_pod")
  page_books=[]
  for book in books_scrap:
    title = (book.h3.a["title"])
    price = book.find("p", class_="price_color").text
    avail = book.find("p", class_="instock availability").text.strip()
    rating = book.find("p", class_="star-rating")["class"][1]
    page_books.append({"Title":title, "Price":price, "Availabilty":avail, "Rating":rating})

url = "https://books.toscrape.com/catalogue/page-1.html"
books=[]
while url:
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")

  books.extend(scrap_books(soup))

  next_button = soup.find("li" , class_="next")
  if next_button:
    next_page = next_button.a["href"]
    url = urljoin(url, next_page)
  else:
    url = None

df = pd.DataFrame(books)
df.to_csv("books.csv", index=False)

df.head()


200
{'Date': 'Wed, 10 Sep 2025 15:34:56 GMT', 'Content-Type': 'text/html', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Last-Modified': 'Wed, 08 Feb 2023 21:02:32 GMT', 'ETag': 'W/"63e40de8-c85e"', 'Strict-Transport-Security': 'max-age=0; includeSubDomains; preload', 'Content-Encoding': 'br'}


,Title,Price,Availabilty,Rating
0,A Light in the Attic,Â£51.77,In stock,Three
1,Tipping the Velvet,Â£53.74,In stock,One
2,Soumission,Â£50.10,In stock,One
3,Sharp Objects,Â£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five


Q2. Write a Python program to scrape the IMDB Top 250 Movies list
(https://www.imdb.com/chart/top/) . For each movie, extract the following details:
1. Rank (1–250)
2. Movie Title
3. Year of Release
4. IMDB Rating
Store the results in a Pandas DataFrame and export it to a CSV file named imdb_top250.csv

In [1]:
!pip install selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# Configure Selenium options
chrome_opts = Options()
chrome_opts.add_argument("--headless")
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")

# Spoof user-agent to avoid 403 Forbidden
chrome_opts.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/115.0.0.0 Safari/537.36"
)

# Launch browser
browser = webdriver.Chrome(options=chrome_opts)

# Open IMDb Top 250
browser.get("https://www.imdb.com/chart/top/")
time.sleep(5)  # wait for page to load

film_list = []
movie_cards = browser.find_elements(By.CSS_SELECTOR, ".ipc-metadata-list-summary-item")

# Extract movie details
for rank, card in enumerate(movie_cards, start=1):
    try:
        name = card.find_element(By.CSS_SELECTOR, "h3").text
        release_year = card.find_element(By.CSS_SELECTOR, ".cli-title-metadata-item").text
        score = card.find_element(By.CSS_SELECTOR, ".ipc-rating-star--imdb").text.split()[0]
        film_list.append([rank, name, release_year, score])
    except Exception as e:
        print(f"Skipping a card due to error: {e}")

browser.quit()

# Save as DataFrame
imdb_table = pd.DataFrame(film_list, columns=["Position", "Movie", "Release Year", "Rating"])
imdb_table.to_csv("imdb_top250.csv", index=False)
print(imdb_table.head())

   Position                        Movie Release Year Rating
0         1  1. The Shawshank Redemption         1994    9.3
1         2             2. The Godfather         1972    9.2
2         3           3. The Dark Knight         2008    9.1
3         4     4. The Godfather Part II         1974    9.0
4         5              5. 12 Angry Men         1957    9.0


Q3. Write a Python program to scrape the weather information for top world cities from the
given website (https://www.timeanddate.com/weather/) . For each city, extract the following
details:
1. City Name
2. Temperature
3. Weather Condition (e.g., Clear, Cloudy, Rainy, etc.)
Store the results in a Pandas DataFrame and export it to a CSV file named weather.csv.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.timeanddate.com/weather"
headers = {"User-Agent": "Mozilla/5.0"}
res = requests.get(url, headers=headers)

soup = BeautifulSoup(res.text, "html.parser")

movies = []
for row in soup.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) >= 4:
        city = cols[0].get_text(strip=True)


        condition = None
        if cols[2].find("img"):
            condition = cols[2].find("img")["alt"]

        temp = cols[3].get_text(strip=True)

        movies.append({"City": city, "Temperature": temp, "Condition": condition})

df = pd.DataFrame(movies)
df.to_csv("weather.csv", index=False)

print(df.head(10))

          City Temperature                         Condition
0        Accra       81 °F             Passing clouds. Warm.
1  Addis Ababa       59 °F              Broken clouds. Cool.
2     Adelaide       49 °F                           Chilly.
3      Algiers       81 °F           Scattered clouds. Warm.
4       Almaty       70 °F              Partly cloudy. Mild.
5        Amman       84 °F       Duststorm. Pleasantly warm.
6   Amsterdam*       68 °F             Passing clouds. Mild.
7       Anadyr       46 °F             Passing clouds. Cool.
8   Anchorage*       51 °F              Mostly cloudy. Cool.
9       Ankara       81 °F  Passing clouds. Pleasantly warm.
